# Estamos em Produção

### Verificar a escala das falhas utilizando o Gradiente Boost Regressor

#### 1 - Carregar o modelo treinado
#### 2 - Carregar os dados a serem inferidos
#### 3 - Fazer as inferências
#### 4 - FIM

In [9]:
# Carregando modelo

import pickle
model = pickle.load(open('model_GBT.sav', 'rb'))

In [10]:
import pandas as pd

df = pd.read_excel('O_G_Equipment_Data.xlsx')
df.head(2)

,Cycle,Preset_1,Preset_2,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Fail
0,1,3,6,44.235186,47.657254,46.441769,64.820327,66.454520,44.483250,False
1,2,2,4,60.807234,63.172076,62.005951,80.714431,81.246405,60.228715,False


In [11]:
# Codificando a coluna Fail

labels = df['Fail'].astype('category').cat.categories.tolist()
replace_Fail = {'Fail': {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}

print(replace_Fail)

{'Fail': {False: 1, True: 2}}


In [12]:
df.replace(replace_Fail, inplace=True)

In [13]:
# Trecho de código utilizado para realizar inferências somente com um novo dataset que apresentará as falhas originadas

df_WF = df.copy()

In [14]:
# Removendo as colunas para realizar a predição 

df = df_WF.drop(columns={'Cycle', 'Preset_1','Preset_2','Fail'})
df.head(4)

,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency
0,44.235186,47.657254,46.441769,64.820327,66.454520,44.483250
1,60.807234,63.172076,62.005951,80.714431,81.246405,60.228715
2,79.027536,83.032190,82.642110,98.254386,98.785196,80.993479
3,79.716242,100.508634,122.362321,121.363429,118.652538,80.315567


In [15]:
# Inferência

inferences = model.predict(df)
df['Resultado'] = inferences
df.head(4)

,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Resultado
0,44.235186,47.657254,46.441769,64.820327,66.454520,44.483250,1.012712
1,60.807234,63.172076,62.005951,80.714431,81.246405,60.228715,1.012712
2,79.027536,83.032190,82.642110,98.254386,98.785196,80.993479,1.012712
3,79.716242,100.508634,122.362321,121.363429,118.652538,80.315567,1.012712


In [16]:
df_t = pd.DataFrame(df.Resultado.value_counts().sort_index())
df_t = df_t.reset_index()
df_t = df_t.rename({'index': 'Fail_Category'}, axis=1)
df_t

,Fail_Category,Resultado
0,1.000000,12
1,1.012712,575
2,1.174242,176
3,1.827586,37


## CATEGORIZANDO AS FALHAS, CRIANDO UM GRAU DE INTENSIDADE E/OU CRITICIDADE PARA A FALHA DETECTADA.

In [17]:
df_WF.drop(df_WF[(df_WF['Fail'] == 1)].index, inplace=True)
df_WF.head(4)

,Cycle,Preset_1,Preset_2,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Fail
12,13,1,1,79.806509,103.129738,120.753103,134.500905,115.873197,76.562856,2
156,157,3,1,91.695332,85.345796,99.890670,95.328429,124.970301,84.377355,2
157,158,2,5,88.916114,88.231887,85.140550,123.711911,88.852872,100.229131,2
158,159,3,5,88.768766,99.228691,89.109514,124.934221,84.306467,88.218162,2


In [18]:
df_2 = df_WF.drop(columns={'Cycle', 'Preset_1','Preset_2','Fail'})
df_2.head(4)

,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency
12,79.806509,103.129738,120.753103,134.500905,115.873197,76.562856
156,91.695332,85.345796,99.890670,95.328429,124.970301,84.377355
157,88.916114,88.231887,85.140550,123.711911,88.852872,100.229131
158,88.768766,99.228691,89.109514,124.934221,84.306467,88.218162


In [19]:
# Inferência

inferences = model.predict(df_2)
df_2['Resultado'] = inferences
df_2.head(4)

,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Resultado
12,79.806509,103.129738,120.753103,134.500905,115.873197,76.562856,1.827586
156,91.695332,85.345796,99.890670,95.328429,124.970301,84.377355,1.012712
157,88.916114,88.231887,85.140550,123.711911,88.852872,100.229131,1.174242
158,88.768766,99.228691,89.109514,124.934221,84.306467,88.218162,1.012712


In [20]:
df_f = pd.DataFrame(df_2.Resultado.value_counts().sort_index())
df_f = df_f.reset_index()
df_f = df_f.rename({'index': 'Fail_Category'}, axis=1)
df_f

,Fail_Category,Resultado
0,1.012712,8
1,1.174242,30
2,1.827586,28
